In [1]:
pip install -r requirements.txt

Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 23.0.1 -> 24.0
[notice] To update, run: python.exe -m pip install --upgrade pip


# Import

In [28]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from matplotlib import colors
import seaborn as sns

import charset_normalizer
import fuzzywuzzy
from fuzzywuzzy import process
import os
import re
from dateutil import parser

# from scipy import stats
# from tabulate import tabulate
# import sklearn
# from sklearn.preprocessing import LabelEncoder
# from sklearn.preprocessing import StandardScaler
# from sklearn.decomposition import PCA
# from sklearn.neighbors import NearestNeighbors
# from sklearn.cluster import AgglomerativeClustering, KMeans, DBSCAN
# from sklearn.metrics import silhouette_score, davies_bouldin_score,calinski_harabasz_score
# from yellowbrick.cluster import KElbowVisualizer
# from scipy.cluster.hierarchy import dendrogram, linkage

import warnings
warnings.filterwarnings('ignore')
plt.style.use('https://github.com/dhaitz/matplotlib-stylesheets/raw/master/pitayasmoothie-light.mplstyle')

# Character encoding

In [3]:
df_name = []
encoding = []
path = []

dataset_folder = "dataset"
for file in os.listdir(dataset_folder):
  file_path = os.path.join(dataset_folder, file)
  path.append(file_path)
  # look at the first ten thousand bytes to guess the character encoding
  with open(file_path, 'rb') as rawdata:
    result = charset_normalizer.detect(rawdata.read())
    encoding.append(result['encoding'])
  # check what the character encoding might be
  file_name = os.path.splitext(file)[0]
  df_name.append(file_name)
  print(f"{file_name}: {result}")

context: {'encoding': 'ascii', 'language': 'English', 'confidence': 1.0}
mobile_plan_attr: {'encoding': 'ascii', 'language': 'English', 'confidence': 1.0}
mobile_plan_user: {'encoding': 'ascii', 'language': 'English', 'confidence': 1.0}
user: {'encoding': 'utf-8', 'language': 'English', 'confidence': 1.0}


In [4]:
data_source = (tuple(df_name), tuple(encoding), tuple(path))

In [5]:
context = pd.read_csv(data_source[2][0], encoding=data_source[1][0])
context.sample(5)

,id,purpose,go_with,weather,time,viettel_no_0,viettel_no_1,viettel_no_2,to_hanoi,to_other,score,direction
1459,16872,Work,!A!lone,Sunny,7 AM,1,0,0,1,0,0,1
10840,5640,Work,Al !!one,S~!~!nowy,7:00,1,0,0,1,0,0,1
6238,4988,Travel,?Frie %nd(s),&%Sunny,20:00,1,1,0,0,1,11,0
1560,6084,Travel,Frien~d%(s),S!!!unny,5 PM,1,1,0,0,1,0,0
10005,14033,Visit,F& am%ily,Sunny,16:00,1,0,0,1,0,0,1


In [6]:
mobile_plan_attr = pd.read_csv(data_source[2][1], encoding=data_source[1][1], delimiter=';')
mobile_plan_attr.sample(5)

,mobile_plan,description,price,duration
4,DATACALL,"2GB/ day, high speed\n300 mins call for extern...",200000,5d
3,SOCIALMEDIAGOLD,"3GB/ day, high speed\nUnlimited for Tik Tok, F...",250000,3d
1,DATAGOLD,"5GB/ day, high speed",200000,5d
2,SOCIALMEDIA,"1GB/ day\nUnlimited for Tik Tok, Facebook, You...",150000,3d
0,DATASILVER,"2GB/ day, high speed",100000,5d


In [7]:
mobile_plan_user = pd.read_csv(data_source[2][2], encoding=data_source[1][2])
mobile_plan_user.sample(5)

,id,mobile_plan,accept
40312,8147.0,DATAGOLD,0.0
32677,18035.0,DATAGOLD,0.0
32365,1681.0,DATAGOLD,0.0
37938,8691.0,SOCIALMEDIAGOLD,1.0
7347,17853.0,SOCIALMEDIA,0.0


In [8]:
user = pd.read_csv(data_source[2][3], encoding=data_source[1][3])
user.sample(5)

,id,name,gender,age,education,profession,income,living_with,nation,phone,job,fb_freq,yt_freq,insta_freq,use_less_than_2GB,use_2GB_to_4GB
4775,9166,안은경,Female,29,Bachelor,Computer & Mathematical,348100$,Ma rr ied_1,KOREA,070-7533-5055,간판 제작 및 설치원,2,3,4,7,1
10664,7389,Marcus Bass,Male,44,Bachelor,Arts Design Entertainment Sports & Media,1212500000 VND,Marr ied_2,US,001-643-804-3123x512,Curator,9,1,1,7,2
4480,6183,하중수,Male,47,Masters,Management,2475000000 VND,Married_2,KOREA,016-605-1245,컴퓨터 보안 전문가,0,3,2,0,2
2874,8450,Алла Сергеевна Беспалова,Female,28,NaN,Student,25500$,U nma rried_0,RUSSIA,+7 145 841 7788,Нотариус,0,0,2,1,0
8900,3501,장경숙,Female,26,Bachelor,Unemployed,8700$,S ingle_0,KOREA,016-640-6135,정보통신관련 관리자,7,3,12,8,1


# Cleaning
## Context dataframe

In [11]:
context.sample(5)

,id,purpose,go_with,weather,time,viettel_no_0,viettel_no_1,viettel_no_2,to_hanoi,to_other,score,direction
6311,8089,Visit,?~%&Alone,Su&n~ny,16:00,1,0,0,0,1,19,0
61,6679,Travel,Frie%!n?~d(s),!S!u!nny,13:00,1,0,0,0,1,0,0
4050,5654,Travel,Friend !(s),!??Snowy,10 PM,1,1,0,0,1,0,0
9902,1168,Travel,Fa? ~mily,Rainy,20:00,1,1,0,0,1,0,0
9949,17211,Visit,Alone,!%S??nowy,5 PM,1,1,0,0,1,0,0


In [12]:
for x in ['go_with', 'weather']:
  # Eliminate special characters and white spaces
  context[x] = context[x].apply(lambda x: re.sub(r'[^a-zA-Z\s]', '', x))
  
  # Lowercase all words
  context[x] = context[x].str.lower()
  
  print(context[x].unique())

['alone' 'friends' 'fa mily' 'family' 'fami ly' 'a l one' 'fr iends'
 ' alone' 'a lone' '  alone' ' friends' 'f riends' 'f rie nds' 'frie nds'
 'frien ds' 'f amily' ' family' 'alo ne' 'friend s' 'al one' 'fri ends'
 ' a lone' ' fri ends' 'f riend s' 'alon e' 'frien  ds' ' fri e nds'
 'friends ' 'a  lone' 'fa  mily' 'al  one' '   alone' 'fri e nds'
 ' al one' 'fam ily' '  family' 'f rien ds' '  a lone' ' f amily'
 'frie   nds' 'fri en ds' 'f a mily' ' frie nds' 'f  amily' 'fr  iends'
 ' fa mily' 'famil y' ' fr iends' 'fr ie nds' 'fr ien ds' ' f  amily'
 'fam  ily' ' frien ds' 'frie nd s' 'frie n ds' ' friend s' 'frie  nds'
 'fr i ends' 'fri  ends' 'f ri ends' ' a  lone' ' alo ne' 'friend  s'
 'a   lone' '  friends' '   friends' ' fam ily' 'f r iends' 'alo  ne'
 '  a  lone' ' fa  mily' 'f r  iends' 'f  riends' ' frie  nds' 'fri end s'
 ' fami ly' 'f rie  nds' ' f riends' ' al  one' 'f a  mily' ' fa m ily'
 '    alone' 'f  ri ends' 'a lo ne' 'frien d s' 'f am ily' '  f riends'
 '   family

In [13]:
# List of possible values for the column 'go_with'
choices = ['alone', 'friend(s)', 'family']

# Apply the fuzzy matching to the column 'go_with'
def correct_name(name):
    return process.extractOne(name, choices)[0]

context['go_with'] = context['go_with'].apply(correct_name)

In [32]:
# Eliminate white spaces
context['time'] = context['time'].str.replace(r'\s+', '', regex=True)

# Function to convert time to 24-hour format
def convert_to_24hr_format(time_str):
    # Use dateutil.parser to parse the time string
    return parser.parse(time_str).strftime('%H:%M')

# Apply the function to the column 'time'
context['time'] = context['time'].apply(convert_to_24hr_format)
context['time'] = pd.to_datetime(context['time'], format='%H:%M').dt.time

In [14]:
context.sample(5)

,id,purpose,go_with,weather,time,viettel_no_0,viettel_no_1,viettel_no_2,to_hanoi,to_other,score,direction
4501,8726,Travel,friend(s),sunny,5 PM,1,0,0,0,1,0,0
1996,4635,Work,alone,sunny,7 AM,1,1,1,0,1,0,0
6357,6950,Travel,family,snowy,9AM,1,0,0,0,1,0,0
10565,13067,Visit,alone,snowy,22:00,1,1,1,0,1,0,0
9720,19741,Travel,alone,sunny,15:00,1,0,0,0,1,0,0


## User dataframe

In [15]:
user.sample(5)

,id,name,gender,age,education,profession,income,living_with,nation,phone,job,fb_freq,yt_freq,insta_freq,use_less_than_2GB,use_2GB_to_4GB
4604,17987,Ермил Якубович Елисеев,Male,28,Bachelor,Computer & Mathematical,4572500000 VND,Married_0,RUSSIA,+7 647 281 71 02,Распиловщик,8,12,1,1,0
11551,9918,정영환,Male,31,Bachelor,Business & Financial,9917500000 VND,Marri ed_2,KOREA,064-703-9273,기타 판매관련 단순 종사원,1,0,5,12,0
3638,9122,William Hunter,Male,28,Bachelor,Management,1510000000 VND,Married_2,AUSTRALIA,0472 986 589,"Surveyor, building control",0,0,1,3,0
2959,17050,朱梅,Female,44,Bachelor,Education&Training&Library,30300$,Ma rried_4,CHINA,15271079787,审计专员/助理,0,3,3,2,3
4618,15213,Michael Evans,Male,37,Masters,Computer & Mathematical,116100$,M ar ried_1,US,001-484-555-1370x204,Teaching laboratory technician,0,0,0,1,1


In [21]:
# Eliminate white spaces
user['living_with'] = user['living_with'].str.strip()
user['living_with'] = user['living_with'].str.replace(r'\s+', '', regex=True)

In [23]:
user.sample(5)

,id,name,gender,age,education,profession,income,living_with,nation,phone,job,fb_freq,yt_freq,insta_freq,use_less_than_2GB,use_2GB_to_4GB
4584,10135,鞠军,Male,35,Masters,Business & Financial,77300$,Married_0,CHINA,18019179427,单证员,2,5,0,5,0
2083,4523,Anna Farley,Female,27,NaN,Food Preparation & Serving Related,710000000 VND,Single_0,ENGLAND,+1-576-286-9005,Immunologist,3,1,2,2,1
5296,19748,Christopher Lopez,Other,40,Masters,Healthcare Practitioners & Technical,2315000000 VND,Married_1,AUSTRALIA,99270731,"Nurse, mental health",3,6,4,8,1
7016,16532,徐婷婷,Male,57,Bachelor,Retired,100900$,Unmarried_0,CHINA,14581222013,促销经理,0,0,2,10,0
77,15209,Billy Ellis,Male,27,Bachelor,Student,61100$,Unmarried_0,US,890.833.5642,Building surveyor,3,18,5,1,0


In [34]:
user.income.unique()

array(['39100$', '41000$', '44300$', ..., '10070000000 VND', '189100$',
       '354200$'], dtype=object)

# Data type check

In [35]:
dataset = [context, mobile_plan_attr, mobile_plan_user, user]
for x in dataset:
  print('-------------------------')
  print(x.dtypes)

-------------------------
id               int64
purpose         object
go_with         object
weather         object
time            object
viettel_no_0     int64
viettel_no_1     int64
viettel_no_2     int64
to_hanoi         int64
to_other         int64
score            int64
direction        int64
dtype: object
-------------------------
mobile_plan    object
description    object
price           int64
duration       object
dtype: object
-------------------------
id             float64
mobile_plan     object
accept         float64
dtype: object
-------------------------
id                    int64
name                 object
gender               object
age                   int64
education            object
profession           object
income               object
living_with          object
nation               object
phone                object
job                  object
fb_freq               int64
yt_freq               int64
insta_freq            int64
use_less_than_2GB     int64
u

In [1]:
#fkasf